In [1]:
from __future__ import division

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import OneClassSVM
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
# Input data files are available in the "../data/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../data"]).decode("utf8"))


test.csv
train.csv



In [2]:
# Any results you write to the current directory are saved as output.
# load data
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')

# remove constant columns
remove = []
for col in df_train.columns:
    if df_train[col].std() == 0:
        remove.append(col)

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

# remove duplicated columns
remove = []
c = df_train.columns
for i in range(len(c)-1):
    v = df_train[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,df_train[c[j]].values):
            remove.append(c[j])

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

y_train = df_train['TARGET'].values
X_train = df_train.drop(['ID','TARGET'], axis=1).values

id_test = df_test['ID']
X_test = df_test.drop(['ID'], axis=1).values

# length of dataset
len_train = len(X_train)
len_test  = len(X_test)

# Good XGBoost Function From Blog Analytics Vidhya

In [6]:
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

def modelfit(alg, X_train, y_train, useTrainCV=True, cv_folds=5, early_stopping_rounds=50, show_progress=True):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X_train, label=y_train)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                          metrics=['auc'], early_stopping_rounds=early_stopping_rounds, show_progress=show_progress)
        alg.set_params(n_estimators=cvresult.shape[0])
        print cvresult
    #Fit the algorithm on the data
    ###alg.fit(X_train, y_train,eval_metric='auc')
        
    #Predict training set:
    ###dtrain_predictions = alg.predict(X_train)
    ###dtrain_predprob = alg.predict_proba(X_train)[:,1]
        
    #Print model report:
    ###print "\nModel Report"
    ###print "Accuracy : %.4g" % metrics.accuracy_score(y_train, dtrain_predictions)
    ###print "AUC Score (Train): %f" % metrics.roc_auc_score(y_train, dtrain_predprob)
                    
    #feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    #feat_imp.plot(kind='bar', title='Feature Importances')
    #plt.ylabel('Feature Importance Score')

# Start With some params and tune n_estimators first of all

In [12]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=(len(y_train)-sum(y_train))/sum(y_train),
 seed=27)

In [13]:
modelfit(xgb1, X_train, y_train)

Will train until cv error hasn't decreased in 50 rounds.
[0]	cv-test-auc:0.7889444+0.0347440533536	cv-train-auc:0.8015272+0.0279617965045
[1]	cv-test-auc:0.8126236+0.00975955848592	cv-train-auc:0.8310436+0.00203585250939
[2]	cv-test-auc:0.8161628+0.00540042233163	cv-train-auc:0.8345774+0.00851164321621
[3]	cv-test-auc:0.821051+0.00857446299193	cv-train-auc:0.839839+0.00757509543174
[4]	cv-test-auc:0.8265596+0.00702946992596	cv-train-auc:0.84461+0.00398633199822
[5]	cv-test-auc:0.829013+0.00750894328118	cv-train-auc:0.8467768+0.00187871726452
[6]	cv-test-auc:0.8305522+0.0069883866636	cv-train-auc:0.8483712+0.00114111005604
[7]	cv-test-auc:0.8312192+0.00737279353841	cv-train-auc:0.8490468+0.000940783375703
[8]	cv-test-auc:0.8312028+0.00746654729845	cv-train-auc:0.849434+0.00116553129516
[9]	cv-test-auc:0.8309764+0.00817816619053	cv-train-auc:0.8491124+0.00157992412476
[10]	cv-test-auc:0.8315886+0.00777993266295	cv-train-auc:0.8500682+0.0011168954114
[11]	cv-test-auc:0.830684+0.0070656777

KeyboardInterrupt: 

In [10]:
xgb1.n_estimators

31

#2. Tune Max_Depth and min_Child_Weight

In [11]:
param_test1 = {'max_depth':range(3,10,2),'min_child_weight':range(1,6,2)}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=31, max_depth=5,
                                                  min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                 objective= 'binary:logistic', nthread=4, scale_pos_weight=(len(y_train)-sum(y_train))/sum(y_train)
                                                  , seed=27), 
                                                 param_grid = param_test1, scoring='roc_auc',n_jobs=2,iid=False, cv=5, 
                                                verbose=1000)
gsearch1.fit(X_train,y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Memmaping (shape=(76020, 306), dtype=float64) to new file /var/folders/rf/x5jhln3158n6pyl8jpk3_psc0000gn/T/joblib_memmaping_pool_30946_4328736208/30946-4483234704-4355148176-0.pkl
Pickling array (shape=(76020,), dtype=int64).
Pickling array (shape=(60815,), dtype=int64).
Pickling array (shape=(15205,), dtype=int64).
Memmaping (shape=(76020, 306), dtype=float64) to old file /var/folders/rf/x5jhln3158n6pyl8jpk3_psc0000gn/T/joblib_memmaping_pool_30946_4328736208/30946-4483234704-4355148176-0.pkl
Pickling array (shape=(76020,), dtype=int64).
Pickling array (shape=(60815,), dtype=int64).
Pickling array (shape=(15205,), dtype=int64).
Memmaping (shape=(76020, 306), dtype=float64) to old file /var/folders/rf/x5jhln3158n6pyl8jpk3_psc0000gn/T/joblib_memmaping_pool_30946_4328736208/30946-4483234704-4355148176-0.pkl
Pickling array (shape=(76020,), dtype=int64).
Pickling array (shape=(60816,), dtype=int64).
Pickling array (shape=(15204,),

KeyboardInterrupt: 

In [ ]:
# searching for more optimum values

In [11]:
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[2,3,4]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=77, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_train,y_train)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: 0.83977, std: 0.00903, params: {'max_depth': 4, 'min_child_weight': 2},
  mean: 0.83993, std: 0.00879, params: {'max_depth': 4, 'min_child_weight': 3},
  mean: 0.83939, std: 0.00928, params: {'max_depth': 4, 'min_child_weight': 4},
  mean: 0.83995, std: 0.00866, params: {'max_depth': 5, 'min_child_weight': 2},
  mean: 0.84055, std: 0.00855, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.84012, std: 0.00910, params: {'max_depth': 5, 'min_child_weight': 4},
  mean: 0.83859, std: 0.00946, params: {'max_depth': 6, 'min_child_weight': 2},
  mean: 0.83917, std: 0.00824, params: {'max_depth': 6, 'min_child_weight': 3},
  mean: 0.84022, std: 0.00877, params: {'max_depth': 6, 'min_child_weight': 4}],
 {'max_depth': 5, 'min_child_weight': 3},
 0.84054568210341452)

#3. Tune Gamma

In [12]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=77, max_depth=5,
 min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train,y_train)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: 0.84055, std: 0.00855, params: {'gamma': 0.0},
  mean: 0.84008, std: 0.00864, params: {'gamma': 0.1},
  mean: 0.83951, std: 0.00919, params: {'gamma': 0.2},
  mean: 0.83993, std: 0.00896, params: {'gamma': 0.3},
  mean: 0.83969, std: 0.00898, params: {'gamma': 0.4}],
 {'gamma': 0.0},
 0.84054568210341452)

This shows that our original value of gamma, i.e. 0 is the optimum one. Before proceeding, a good idea would be to re-calibrate the number of boosting rounds for the updated parameters.

In [14]:
xgb2 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=3,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb2, X_train, y_train,early_stopping_rounds=75)

Will train until cv error hasn't decreased in 75 rounds.
Stopping. Best iteration: 74


    test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0        0.765040      0.047673        0.775638       0.045115
1        0.804597      0.014327        0.815781       0.006890
2        0.808440      0.017670        0.820612       0.007208
3        0.812558      0.018928        0.824634       0.011052
4        0.814372      0.014797        0.827276       0.005470
5        0.815404      0.011688        0.829150       0.005696
6        0.817367      0.010937        0.831988       0.007426
7        0.820128      0.009868        0.835253       0.004960
8        0.820196      0.010866        0.836122       0.003272
9        0.822151      0.009638        0.838940       0.002889
10       0.825126      0.007775        0.841629       0.002382
11       0.826819      0.007185        0.843270       0.002261
12       0.826916      0.008019        0.843412       0.000861
13       0.828277      0.007417        0.845159       0.001112
14       0.828938      0.007303        0.845927       0

Nothing awesome got. Will use the same no of trees as was using previously

#Step 4: Tune subsample and colsample_bytree

In [15]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=77, max_depth=5,
 min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_train,y_train)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: 0.83992, std: 0.00870, params: {'subsample': 0.6, 'colsample_bytree': 0.6},
  mean: 0.84032, std: 0.00879, params: {'subsample': 0.7, 'colsample_bytree': 0.6},
  mean: 0.84023, std: 0.00912, params: {'subsample': 0.8, 'colsample_bytree': 0.6},
  mean: 0.84030, std: 0.00912, params: {'subsample': 0.9, 'colsample_bytree': 0.6},
  mean: 0.83913, std: 0.00859, params: {'subsample': 0.6, 'colsample_bytree': 0.7},
  mean: 0.83957, std: 0.00821, params: {'subsample': 0.7, 'colsample_bytree': 0.7},
  mean: 0.83974, std: 0.00924, params: {'subsample': 0.8, 'colsample_bytree': 0.7},
  mean: 0.84039, std: 0.00866, params: {'subsample': 0.9, 'colsample_bytree': 0.7},
  mean: 0.83894, std: 0.00921, params: {'subsample': 0.6, 'colsample_bytree': 0.8},
  mean: 0.83945, std: 0.00874, params: {'subsample': 0.7, 'colsample_bytree': 0.8},
  mean: 0.84055, std: 0.00855, params: {'subsample': 0.8, 'colsample_bytree': 0.8},
  mean: 0.84058, std: 0.00920, params: {'subsample': 0.9, 'colsample_bytree'

In [ ]:
#0.05 intervals around these

In [16]:
param_test5 = {
 'subsample':[i/100.0 for i in range(85,100,5)],
 'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=77, max_depth=5,
 min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(X_train,y_train)
gsearch5.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: 0.83946, std: 0.00911, params: {'subsample': 0.85, 'colsample_bytree': 0.75},
  mean: 0.83968, std: 0.00908, params: {'subsample': 0.9, 'colsample_bytree': 0.75},
  mean: 0.83971, std: 0.00937, params: {'subsample': 0.95, 'colsample_bytree': 0.75},
  mean: 0.83972, std: 0.00886, params: {'subsample': 0.85, 'colsample_bytree': 0.8},
  mean: 0.84058, std: 0.00920, params: {'subsample': 0.9, 'colsample_bytree': 0.8},
  mean: 0.84008, std: 0.00876, params: {'subsample': 0.95, 'colsample_bytree': 0.8},
  mean: 0.83981, std: 0.00893, params: {'subsample': 0.85, 'colsample_bytree': 0.85},
  mean: 0.84041, std: 0.00923, params: {'subsample': 0.9, 'colsample_bytree': 0.85},
  mean: 0.84007, std: 0.00857, params: {'subsample': 0.95, 'colsample_bytree': 0.85}],
 {'colsample_bytree': 0.8, 'subsample': 0.9},
 0.8405757633860711)

# Step 5: Tuning Regularization Parameters

In [17]:
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=77, max_depth=5,
 min_child_weight=3, gamma=0, subsample=0.9, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(X_train,y_train)
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

([mean: 0.84058, std: 0.00920, params: {'reg_alpha': 1e-05},
  mean: 0.84025, std: 0.00924, params: {'reg_alpha': 0.01},
  mean: 0.83985, std: 0.00872, params: {'reg_alpha': 0.1},
  mean: 0.84018, std: 0.00886, params: {'reg_alpha': 1},
  mean: 0.82964, std: 0.00838, params: {'reg_alpha': 100}],
 {'reg_alpha': 1e-05},
 0.84057580892653583)


#Step 6: Reducing Learning Rate



In [29]:
xgb3 = XGBClassifier(missing=np.nan,
 learning_rate =0.03,
 n_estimators=10000,
 max_depth=5,
 min_child_weight=3,
 gamma=0,
 subsample=0.9,
 colsample_bytree=0.8,
 reg_alpha=1e-05,
 objective= 'binary:logistic',
 nthread=4,
 seed=27)
modelfit(xgb3, X_train, y_train,show_progress=True)

Will train until cv error hasn't decreased in 50 rounds.
[0]	cv-test-auc:0.7668878+0.0447758819741	cv-train-auc:0.7764546+0.0421354577979
[1]	cv-test-auc:0.8025282+0.0135798352479	cv-train-auc:0.8141794+0.0045415561474
[2]	cv-test-auc:0.8078562+0.017067371636	cv-train-auc:0.8198564+0.00740330335999
[3]	cv-test-auc:0.810158+0.0160780984323	cv-train-auc:0.8219422+0.00840285323923
[4]	cv-test-auc:0.8127858+0.0134836377198	cv-train-auc:0.8248904+0.00548050211568
[5]	cv-test-auc:0.8125252+0.0137919171459	cv-train-auc:0.824738+0.00683081450487
[6]	cv-test-auc:0.815006+0.0118980251975	cv-train-auc:0.8265252+0.00682582435754
[7]	cv-test-auc:0.8182028+0.0108325162063	cv-train-auc:0.830798+0.00439759384209
[8]	cv-test-auc:0.8180468+0.0108898902364	cv-train-auc:0.8313988+0.00342016230024
[9]	cv-test-auc:0.8190282+0.0104964543995	cv-train-auc:0.8326366+0.00266678567568
[10]	cv-test-auc:0.8203448+0.00962782046779	cv-train-auc:0.8340058+0.00222084338935
[11]	cv-test-auc:0.8215928+0.00928683449621	cv

     test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0         0.766888      0.044776        0.776455       0.042135
1         0.802528      0.013580        0.814179       0.004542
2         0.807856      0.017067        0.819856       0.007403
3         0.810158      0.016078        0.821942       0.008403
4         0.812786      0.013484        0.824890       0.005481
5         0.812525      0.013792        0.824738       0.006831
6         0.815006      0.011898        0.826525       0.006826
7         0.818203      0.010833        0.830798       0.004398
8         0.818047      0.010890        0.831399       0.003420
9         0.819028      0.010496        0.832637       0.002667
10        0.820345      0.009628        0.834006       0.002221
11        0.821593      0.009287        0.835550       0.001886
12        0.821010      0.010944        0.835250       0.002286
13        0.822265      0.010516        0.836431       0.001922
14        0.823030      0.010157        

[257]	cv-test-auc:0.8397236+0.00490025805851	cv-train-auc:0.8806558+0.00038889967858
Stopping. Best iteration: 207


In [30]:
print('Overall AUC:', roc_auc_score(y_train, xgb3.predict_proba(X_train)[:,1]))

# predicting
y_pred= xgb3.predict_proba(X_test)[:,1]

submission = pd.DataFrame({"ID":id_test, "TARGET":y_pred})
submission.to_csv("submission_xgb.csv", index=False)

print('Completed!')

('Overall AUC:', 0.86930937094208349)
Completed!
